In [1]:
import pandas as pd
from tqdm import tqdm

df = pd.read_csv('sentiment140/training.1600000.processed.noemoticon.csv',index_col=None, header=None, encoding='latin-1')

In [2]:
g = df.groupby(by = df[4]).groups

In [3]:
tweets = {}
for user in tqdm(g):
    tweets[user] = df.iloc[g[user]][5].values

100%|████████████████████████████████████████████████████████████████████████| 659775/659775 [04:07<00:00, 2660.81it/s]


In [7]:
def clean(doc):
    # input: string
    # output: string
    from nltk.stem.wordnet import WordNetLemmatizer
    from nltk.corpus import stopwords 
    import string
    lemma = WordNetLemmatizer()
    stop = set(stopwords.words('english'))
    exclude = set(string.punctuation) 
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude or ch == "!")
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split()).split()
    link_free = [word for word in normalized if 'http' not in word]
    cleaned = " ".join(word for word in link_free)
    
    return cleaned

In [5]:
# getting sentiment of sentence
import nltk
import string as st
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
def sentiment(tweets):
    # input: list of strings
    # output: list of ints
    sentiments = []
    for tweet in tweets:
        tweet = clean(tweet)
        score = sid.polarity_scores(tweet)
        sentiments.append(score['compound'])
    return sentiments

C:\ProgramData\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [6]:
sentiments = {}
overall = 0
for user in tqdm(tweets):
    sentiments[user] = sentiment(tweets[user])

100%|█████████████████████████████████████████████████████████████████████████| 659775/659775 [17:04<00:00, 643.96it/s]


In [12]:
s/c

0.15988854418754891

In [11]:
s = 0
c = 0
for i in sentiments:
    s += sum(sentiments[i])
    c += len(sentiments[i])

In [23]:
import csv
with open('random_tweet_sentiments.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['user','tweets','sentiments'])
    for key, value in tweets.items():
       writer.writerow([key, value, sentiments[key]])